# Задача 

Задача практически аналогичная тому, что было в предыдущей лабораторной.

В файлах `airlines.train-1.tsv`, `airlines.train-2.tsv` и `airlines.test.tsv` находятся данные о пользовательских оценках различных авиакомпаний. Полноценный набор данных доступен <a href="https://github.com/quankiquanki/skytrax-reviews-dataset"> по ссылке </a>.

В данных есть информация про название авиакомпании, страну того, кто оставляет отзыв, класс, которым он летел, текстовое сообщение и итоговая оценка от 0 до 10.

Задача - по первым 4 параметрам (авиакомпания, страна, класс, текстовое сообщение) предсказать оценку, которую поставил пользователь. Для этого необходимо дополнительно превратить данные в формат vw. Про формат, в котором нужно предоставить решения будет написано ниже.

В качестве ответа необходимо сдать обученные веса модели vowpal wabbit. Для оценки решения на тестовых данных будет запущен vw с этими весами и будет подсчитана метрика R2. Решения, которые получили качество больше `0.35` будут оцениваться в 100%. Решения с меньшим качеством будут оценены ниже в соответствии с полученных качеством. Саму модель (веса) необходимо сохранить в файл `result.vw`.

Формат vw:
* Целевая переменная - пользовательская оценка
* 4 неймспейса - name, country, cabin, review
* Значения в name, country, cabin приведены в монолитный формат - все символы, не являющиеся буквой или цифрой (то есть подходящие под регулярное выражение `\W`) заменены на `_`, а также вся строка приведена к нижнему регистру.
* В review оставлены только корректные элементы (то есть подходящие под регулярное выражение `[a-zA-Z0-9_]+`).

Для демонстрации того, как выглядит этот формат, в файле `airlines.test.sample.vw` лежат 10 первых элементов из тестовой выборки, которые закодированны соответствующим образом.


В отличии от предыдущей задачи, необходимо обучить модель распределенно, используюя два рабочих, как было показано выше в лабораторной. Для этих двух рабочих необходимо использовать части `airlines.train-1.tsv` и `airlines.train-2.tsv` соответственно.

In [1]:
import pandas as pd
import numpy as np

In [2]:
df_train_1 = pd.read_csv('airlines.train-1.tsv', sep='\t')
df_train_2 = pd.read_csv('airlines.train-2.tsv', sep='\t')
df_test = pd.read_csv('airlines.test.tsv', sep='\t')

In [3]:
df_train_1.head()

,airline_name,author_country,cabin_flown,content,rating
0,sunwing-airlines,Canada,Economy,March 5th 2014 from Ottawa Canada to Cuba WG 6...,9.0
1,lufthansa,United Kingdom,Economy,SIN-FRA-BHX in Economy. First leg from Singapo...,7.0
2,spirit-airlines,United States,Economy,"Spirit does what they state on their web site,...",7.0
3,sunwing-airlines,Canada,Premium Economy,My fiancé and I were booked to fly to Cayo San...,1.0
4,british-airways,United States,First Class,DXB-LHR B777-200ER BA0108 August 18 First Clas...,9.0


In [4]:
df_train_2.head()

,airline_name,author_country,cabin_flown,content,rating
0,ryanair,France,Economy,Unusually this was my first trip with FR for 4...,8.0
1,united-airlines,United States,Business Class,AMS-ORD 11 Jul. I had a most wonderful flight ...,10.0
2,hawaiian-airlines,United States,Economy,LAS-HNL-LIH. Midnight madness flight out of La...,7.0
3,china-eastern-airlines,India,Business Class,MU563 Shanghai to Delhi July 5. They boarded t...,1.0
4,lan-airlines,Ecuador,Economy,UIO-GYE-UIO. First leg: Web check-in worked pe...,8.0


In [5]:
df_test.head()

,airline_name,author_country,cabin_flown,content,rating
0,south-african-airways,United Kingdom,Economy,JNB-LHR on the new airbus. Seats were roomy an...,8.0
1,jet-airways,Qatar,Business Class,Flew Business Class DOH-BOM-DOH. Outbound: Use...,6.0
2,american-airlines,United States,First Class,This is a rough review because we flew first b...,5.0
3,flybe,United Kingdom,Economy,Am thoroughly fed up with Flybe customer servi...,1.0
4,american-airlines,United Arab Emirates,Economy,I have flown MIA-JFK on an old B767-300. Fligh...,5.0


In [6]:
import re
def formating(text):
    return re.sub('\W', '_', text).lower()
formating('United Kingdom')

def cleaning(text):
    word_pattern = re.compile(r"[a-zA-Z0-9_]+")
    words = []
    for match in re.finditer(word_pattern, text.lower()):
        words.append(match.group(0))
    
    if not words: 
        return None
    return " ".join(words)

def df_format(df):
    for column in df.columns[:3]:
        df[column] = df[column].apply(formating)
    
    df[df.columns[3]] = df[df.columns[3]].apply(cleaning)
    return df

def write_vw(df, filename):
    with open(filename, "w") as f:
        for name, country, cabin, review, rating in df.values:
            vw_object = "{} |name {} |country {} |cabin {} |review {}".format(rating, name, country, cabin, review)
            if not vw_object:
                continue
            f.write(vw_object + '\n')

In [7]:
write_vw(df_format(df_train_1), "airlines_train_1.vw")
write_vw(df_format(df_train_2), "airlines_train_2.vw")
write_vw(df_format(df_test), "airlines_test.vw")

In [8]:
! head -n 2 airlines_test.vw

8.0 |name south_african_airways |country united_kingdom |cabin economy |review jnb lhr on the new airbus seats were roomy and comfy staff polite and friendly and inflight entertainment system outstanding we had terrible turbulence throughout the flight but the captain was informative and reassuring and everyone remained calm food not great but otherwise excellent
6.0 |name jet_airways |country qatar |cabin business_class |review flew business class doh bom doh outbound used the oryx lounge at doha airport which was nice cabin was nearly empty seats are similar to those on jet s domestic business class found it difficult to sleep with the recline provided at 6 3 legrests did not help as my legs overshot it the light sandwich was passable service was attentive and cheerful inbound evening flight so looked forward to meal and wine same cheap french table wine indian non veg meal was not great cabin crew were attentive and friendly ife was limited one negative was that my bag was one of th

In [9]:
! head -n 2 airlines.test.sample.vw

8.0 |name south_african_airways |country united_kingdom |cabin economy |review jnb lhr on the new airbus seats were roomy and comfy staff polite and friendly and inflight entertainment system outstanding we had terrible turbulence throughout the flight but the captain was informative and reassuring and everyone remained calm food not great but otherwise excellent
6.0 |name jet_airways |country qatar |cabin business_class |review flew business class doh bom doh outbound used the oryx lounge at doha airport which was nice cabin was nearly empty seats are similar to those on jet s domestic business class found it difficult to sleep with the recline provided at 6 3 legrests did not help as my legs overshot it the light sandwich was passable service was attentive and cheerful inbound evening flight so looked forward to meal and wine same cheap french table wine indian non veg meal was not great cabin crew were attentive and friendly ife was limited one negative was that my bag was one of th

In [10]:
%%bash --bg --out OUT --err ERR
spanning_tree --nondaemon

In [11]:
! ps aux | grep spanning_tree

jovyan       186  0.0  0.0   6064  1244 ?        S    10:18   0:00 spanning_tree --nondaemon
jovyan       187  0.0  0.0   6892  2096 pts/0    Ss+  10:18   0:00 /bin/bash -c  ps aux | grep spanning_tree
jovyan       189  0.0  0.0   6432   548 pts/0    S+   10:18   0:00 grep spanning_tree


In [12]:
%%bash --bg --out OUT --err ERR
vw --span_server localhost --total 2 --node 0 --unique_id 1 -d airlines_train_1.vw --learning_rate 0.1 --bit_precision 22 --passes 129 --cache_file vw.cache1

In [13]:
%%time
! vw --span_server localhost --total 2 --node 1 --unique_id 1 -d airlines_train_2.vw -f result.vw --learning_rate 0.1 --bit_precision 22 --passes 129 --cache_file vw.cache2

final_regressor = result.vw
Num weight bits = 22
learning rate = 0.1
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
using cache_file = vw.cache2
ignoring text input in favor of cache input
num sources = 1
Enabled reductions: gd, scorer
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
64.000000 64.000000            1            1.0   8.0000   0.0000      237
76.161049 88.322098            2            2.0  10.0000   0.6020      111
46.230893 16.300738            4            4.0   1.0000   1.1863      117
47.609640 48.988387            8            8.0   6.0000   5.0694      334
32.047484 16.485328           16           16.0   1.0000   8.5656      477
30.549394 29.051304           32           32.0   7.0000   2.2265       96
27.594156 24.638918           64           64.0   1.0000   0.9866       45
24.422146 21.250137          128          128.0   5.0000   4.7411      115
22.064454 19.

In [14]:
# При проверки будет запущена примерно следующая команда. 
# Вместо airlines.test.sample.vw будет использоваться целиком airlines.test.tsv переведенный в указанный выше формат
# Так как файл airlines.test.tsv присутствует у вас целиком, после преобразования в vw формат можно использовать его
# для самопроверки

! vw --testonly --initial_regressor result.vw --predictions predictions.txt airlines_test.vw

only testing
predictions = predictions.txt
Num weight bits = 22
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = airlines_test.vw
num sources = 1
Enabled reductions: gd, scorer
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
2.331803 2.331803            1            1.0   8.0000   9.5270       48
2.023103 1.714402            2            2.0   6.0000   7.3094      116
7.635035 13.246968            4            4.0   1.0000   4.2069       92
4.807511 1.979987            8            8.0   8.0000   6.6038      152
4.000725 3.193938           16           16.0   8.0000   5.4146       53
3.356973 2.713221           32           32.0   8.0000   7.3259       56
3.264257 3.171542           64           64.0   8.0000   6.6515       37
3.083323 2.902389          128          128.0   8.0000   9.5909      200
3.593154 4.102986          256          256.0   2.0000   3.

In [15]:
from sklearn.metrics import r2_score

def read_target_from_vw(vw_object):
    return float(vw_object.split(' ')[0])


def calc_r2(predictions_path, answers_path):
    with open(predictions_path, 'r') as f:
        y_pred = np.array([float(value) for value in f.readlines()])
        
    with open(answers_path, 'r') as f:
        y_expected = np.array([read_target_from_vw(value) for value in f.readlines()])
        
    return r2_score(y_expected, y_pred)

In [16]:
# После запуска будет подсчитана метрика r2 на всем тестовом наборе данных. 
# Ваша задача - выбить не меньше 0.35, подстраивая параметры vw


calc_r2('predictions.txt', 'airlines_test.vw')

0.6509455899286707